# loading data and insights 

In [1]:
import pandas as pd
import numpy as np

# Load dataset 
df = pd.read_csv("/kaggle/input/drug-labels-and-side-effects-dataset-1400-records/realistic_drug_labels_side_effects.csv")

In [2]:
# Basic info
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nMissing values per column:\n", df.isnull().sum())


Shape: (1436, 15)

Columns: ['drug_name', 'manufacturer', 'approval_year', 'drug_class', 'indications', 'side_effects', 'dosage_mg', 'administration_route', 'contraindications', 'warnings', 'price_usd', 'batch_number', 'expiry_date', 'side_effect_severity', 'approval_status']

Missing values per column:
 drug_name               0
manufacturer            0
approval_year           0
drug_class              0
indications             0
side_effects            0
dosage_mg               0
administration_route    0
contraindications       0
warnings                0
price_usd               0
batch_number            0
expiry_date             0
side_effect_severity    0
approval_status         0
dtype: int64


In [3]:
# Sample records
print("\nSample rows:")
display(df.sample(5, random_state=42))


Sample rows:


,drug_name,manufacturer,approval_year,drug_class,indications,side_effects,dosage_mg,administration_route,contraindications,warnings,price_usd,batch_number,expiry_date,side_effect_severity,approval_status
594,Rivadipine26,Johnson & Johnson,2017,Antiviral,Pain relief,"Diarrhea, Dry mouth",580,Inhalation,Heart disease,Use with caution in elderly,84.03,XG772Dy,2026-06-14,Moderate,Pending
754,Cefdipine81,Pfizer Inc.,1999,Anti-inflammatory,Fever reduction,Constipation,950,Intravenous,Kidney impairment,Use with caution in elderly,486.71,vG592Kr,2027-06-15,Moderate,Approved
630,Viraprofen54,Roche Holding AG,2013,Antiviral,Pain relief,Insomnia,290,Intravenous,Bleeding disorders,Take with food,238.48,XG870Hl,2025-11-07,Mild,Rejected
1259,Virazole38,GlaxoSmithKline,2002,Anti-inflammatory,Pain relief,"Nausea, Blurred vision, Rash",970,Topical,Diabetes,Take with food,58.89,jz375MU,2028-07-20,Moderate,Approved
903,Zylodipine39,Moderna Therapeutics,1992,Antihistamine,Depression treatment,"Constipation, Diarrhea",140,Inhalation,Allergic reaction,Do not discontinue abruptly,290.37,po625ZS,2025-11-03,Mild,Rejected


In [4]:
# Quick text stats — length of indications & side effects text
df['indications_length'] = df['indications'].astype(str).apply(len)
df['effect_length'] = df['side_effects'].astype(str).apply(len)

print("\nAverage indications text length:", round(df['indications_length'].mean(), 2))
print("Average side effects text length:", round(df['effect_length'].mean(), 2))



Average indications text length: 17.81
Average side effects text length: 18.76


In [5]:
# Quick look at unique counts
print("\nUnique drugs count:", df['drug_name'].nunique())
print("Unique side effects count (raw text):", df['side_effects'].nunique())


Unique drugs count: 1337
Unique side effects count (raw text): 435
